<a href="https://colab.research.google.com/github/davidelgas/DataSciencePortfolio/blob/main/Language_Models/LLM_RAG_Frontend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit

!pip install snowflake

!pip install -q sentence-transformers faiss-cpu pandas openai --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import streamlit as st
import pandas as pd
import snowflake.connector
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI
import os
import json

# Set page configuration
st.set_page_config(
    page_title="BMW E9 Knowledge Base",
    page_icon="🚗",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ======== SIDEBAR: Configuration ========
with st.sidebar:
    st.title("BMW E9 Knowledge Base")
    st.image("https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/BMW_E9_front_20071007.jpg/1280px-BMW_E9_front_20071007.jpg", width=300)
    st.markdown("---")

    # Configuration options
    st.subheader("Configuration")

    # Model selection
    embedding_model = st.selectbox(
        "Embedding Model",
        ["all-MiniLM-L6-v2", "all-mpnet-base-v2"],
        index=0
    )

    llm_model = st.selectbox(
        "Language Model",
        ["gpt-3.5-turbo", "gpt-4"],
        index=0
    )

    # Search parameters
    top_k = st.slider("Number of documents to retrieve", 1, 10, 5)
    temperature = st.slider("Temperature", 0.0, 1.0, 0.2, 0.1)

    # Add a citation toggle
    show_citations = st.toggle("Show Citations", value=True)

    st.markdown("---")
    st.caption("Created by David Elgas")

# ======== FUNCTIONS ========
@st.cache_resource
def load_credentials():
    """Load Snowflake credentials from file"""
    sf_creds_path = '/content/drive/Othercomputers/My Mac/Git/credentials/snowflake_credentials.txt'
    openai_key_path = "/content/drive/Othercomputers/My Mac/CSCI_104/credentials/openaikey.txt"

    # Snowflake credentials
    sf_env = {}
    with open(sf_creds_path, 'r') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                sf_env[key.strip()] = value.strip()

    # OpenAI API key
    with open(openai_key_path, 'r') as file:
        openai_api_key = file.read().strip()

    return sf_env, openai_api_key

@st.cache_resource
def load_data_from_snowflake():
    """Load data from Snowflake database"""
    sf_env, _ = load_credentials()

    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user=sf_env['USER'],
        password=sf_env['PASSWORD'],
        account=sf_env['ACCOUNT'],
        database='E9_CORPUS',
        schema='E9_CORPUS_SCHEMA',
        warehouse='COMPUTE_WH'  # or use your default warehouse
    )

    cur = conn.cursor()
    cur.execute("SELECT THREAD_ID, THREAD_TITLE, THREAD_FIRST_POST, THREAD_ALL_POSTS FROM E9_CORPUS.E9_CORPUS_SCHEMA.E9_FORUM_CORPUS")
    rows = cur.fetchall()

    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=['thread_id', 'thread_title', 'thread_first_post', 'thread_all_posts'])

    # Create a combined text field
    df["full_text"] = (
        df["thread_title"].fillna("") + "\n\n" +
        df["thread_first_post"].fillna("") + "\n\n" +
        df["thread_all_posts"].fillna("")
    )

    return df

@st.cache_resource
def load_embedding_model(model_name):
    """Load the embedding model"""
    return SentenceTransformer(model_name)

@st.cache_resource
def create_faiss_index(df, model):
    """Create FAISS index from corpus embeddings"""
    corpus_embeddings = model.encode(df["full_text"].tolist(), show_progress_bar=True)

    dimension = corpus_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(corpus_embeddings)

    return index, corpus_embeddings

def search_similar_documents(question, index, model, df, top_k=5):
    """Search for similar documents using the FAISS index"""
    question_embedding = model.encode([question])
    distances, indices = index.search(question_embedding, top_k)

    results = []
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        results.append({
            'index': idx,
            'distance': float(dist),
            'thread_id': int(df.iloc[idx]["thread_id"]),
            'title': df.iloc[idx]["thread_title"],
            'first_post': df.iloc[idx]["thread_first_post"],
            'full_text': df.iloc[idx]["full_text"]
        })

    return results

def generate_answer(question, retrieved_docs, with_context=True):
    """Generate answer using OpenAI API"""
    _, openai_api_key = load_credentials()
    client = OpenAI(api_key=openai_api_key)

    if with_context:
        context = "\n\n".join([f"Thread {i+1}:\n{doc['full_text']}" for i, doc in enumerate(retrieved_docs)])
        prompt = f"""You are an expert on BMW E9 maintenance. Use the following forum threads to answer the question.

{context}

Question: {question}
Answer:"""
    else:
        prompt = f"""You are an expert on BMW E9 maintenance.

Question: {question}
Answer:"""

    response = client.chat.completions.create(
        model=llm_model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )

    return response.choices[0].message.content

# ======== MAIN APP ========
# Load data and models
try:
    with st.spinner("Loading data and models..."):
        df = load_data_from_snowflake()
        model = load_embedding_model(embedding_model)
        index, embeddings = create_faiss_index(df, model)
        st.success(f"Loaded {len(df)} forum threads")
except Exception as e:
    st.error(f"Error loading data: {str(e)}")
    st.stop()

# App title and description
st.title("BMW E9 Knowledge Base")
st.markdown("""
This application provides answers to your BMW E9 maintenance and repair questions using data from an E9 forum.
The system uses RAG (Retrieval-Augmented Generation) to find relevant forum posts and generate accurate answers.
""")

# User input
question = st.text_input("Ask a question about BMW E9 maintenance:", "How do I remove the steering wheel in an E9?")

if st.button("Ask Question", type="primary"):
    if not question:
        st.warning("Please enter a question")
    else:
        # Search for relevant documents
        with st.spinner("Searching for relevant information..."):
            results = search_similar_documents(question, index, model, df, top_k)

        # Generate answer with context
        with st.spinner("Generating answer..."):
            answer = generate_answer(question, results)

        # Display answer
        st.markdown("### Answer")
        st.markdown(answer)

        # Display source documents
        if show_citations:
            st.markdown("### Sources")
            for i, result in enumerate(results):
                with st.expander(f"Thread {i+1}: {result['title']} (Distance: {result['distance']:.4f})"):
                    st.markdown(f"**Thread ID:** {result['thread_id']}")
                    st.markdown(f"**First Post:**\n{result['first_post'][:500]}...")

# Add information about the database
st.markdown("---")
with st.expander("About the Database"):
    st.markdown(f"""
    - **Total Threads:** {len(df)}
    - **Embedding Model:** {embedding_model}
    - **Language Model:** {llm_model}
    - **Index Type:** FAISS (Facebook AI Similarity Search)
    """)

    # Display statistics
    if st.checkbox("Show Database Statistics"):
        # Calculate statistics
        avg_title_len = df['thread_title'].str.len().mean()
        avg_posts_len = df['thread_all_posts'].str.len().mean()

        col1, col2 = st.columns(2)
        with col1:
            st.metric("Avg. Title Length", f"{avg_title_len:.1f} chars")
        with col2:
            st.metric("Avg. Post Length", f"{avg_posts_len:.1f} chars")

2025-05-03 20:38:52.867 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 20:38:52.881 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 20:38:53.193 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-03 20:38:53.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 20:38:53.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 20:38:53.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-03 20:38:53.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

NameError: name 'df' is not defined